In [1]:
import pandas as pd
import numpy as np
import graphviz
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, tree
from IPython.display import SVG, Image, HTML

In [2]:
mush = pd.read_csv('D:/mushrooms.csv')

mush = mush.drop(mush.columns[5],axis='columns')

In [3]:
Y_names = np.unique(mush['class']).tolist()
y = mush['class'].values.tolist()

In [4]:
mush = mush.drop(columns=['class'])

In [5]:
col = mush.columns.tolist()
for i in range (len(col)):
    dum = pd.get_dummies(mush[col[i]],prefix=col[i])
    merged =  pd.concat([mush,dum],axis='columns')
    mush = merged.drop([col[i],merged.columns[-1]],axis='columns')

In [6]:
X=mush.values

In [7]:
y = pd.get_dummies(y)
y = y.drop(columns='e',axis='columns')
y = y.values.tolist()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True, stratify=y, test_size=0.2)

In [9]:
log_full = linear_model.LogisticRegression()
log_full.fit(X_train, y_train)

C:\Users\Leonid\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Leonid\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
full_predict = log_full.predict(X_test)
#print(y_test)
#print(full_predict)
print("Accuracy", metrics.accuracy_score(y_test, full_predict))

Accuracy 0.9950769230769231


In [11]:
dt_pruned = tree.DecisionTreeClassifier()
dt_pruned.fit(X_train, y_train)
dt_pred = dt_pruned.predict(X_test)

In [12]:
col = mush.columns.tolist()
dot = graphviz.Source(tree.export_graphviz(dt_pruned, feature_names=col, class_names=Y_names))
svg_bytes = dot.pipe(format='svg')
no_wrap_div = '<div style="width:80% ;height:80% ;">{}</div>'
HTML(no_wrap_div.format(svg_bytes.decode()))

In [13]:
dot.format = 'png'
dot.render('dtree_render',view=True)

'dtree_render.png'

In [14]:
dt_pruned.feature_importances_
importants = {}
for i in range (len(dt_pruned.feature_importances_)):
    if dt_pruned.feature_importances_[i]>0:
        importants[col[i]]=dt_pruned.feature_importances_[i]
from collections import OrderedDict
importants = OrderedDict(sorted(importants.items(), key=lambda x: x[1],reverse=True))
importants        

OrderedDict([('stalk-surface-above-ring_k', 0.34753541860297876),
             ('gill-size_b', 0.3074499972264671),
             ('spore-print-color_h', 0.1251856574071015),
             ('gill-spacing_c', 0.0765642529499571),
             ('cap-surface_f', 0.03791357431997616),
             ('spore-print-color_r', 0.0368482706142081),
             ('stalk-shape_e', 0.02674956843055875),
             ('stalk-surface-below-ring_f', 0.02371086520654151),
             ('habitat_d', 0.01328830139853946),
             ('stalk-color-below-ring_n', 0.004754093843671585)])